In [19]:
import torch
import pandas as pd
from model import *
from utils import *
import numpy as np

In [20]:
data_v8 = pd.read_csv('/Users/sb/Desktop/modeling/SASRec.pytorch/data/V8.csv')
data_v8

,name,restaurant,content,type,address,date,type2,type_dict,name_dict
0,keem,토리아에즈 방이점,맛있어서 몇일만에 재방문이요\n,이자카야,서울특별시 송파구 방이동,11.28.화,주점,1,1
1,keem,토리아에즈 방이점,예전부터 다른 지점 방문하고 싶었는데\n근처에 생겨서 다녀왔어요\n맛있네요!,이자카야,서울특별시 송파구 방이동,11.24.금,주점,1,1
2,keem,송파와인집,매달 추천와인이있어서 좋아요\n간만에 화이트 와인이 땡겨서 화이트2 레드1세트로 구...,주류,서울특별시 송파구 석촌동,11.12.일,주점,2,1
3,keem,명동교자 이태원점,날씨가 추워져서 뜨끈한 칼국수 먹으러 방문했어요\n맛이 변함없이 좋고 마늘김치도 역...,"칼국수,만두",서울특별시 용산구 이태원동,11.12.일,한식,3,1
4,keem,진구정,사장님이 오픈하는곳은 무조건 가봐야져\n평냉에 비냉 제육까지 잘 먹었습니다\n제육에...,이북음식,서울특별시 광진구 구의동,11.11.토,한식,4,1
...,...,...,...,...,...,...,...,...,...
116684,cnswkd,익선잡방,음식도 맛있거 분위기도 좋았어요,브런치,서울특별시 종로구 익선동,22.7.7.목,브런치,37,9526
116685,cnswkd,화포식당 가재울뉴타운점,고기 구워주신분들 너무 친절하시고 맛있게 구워주세요 육즙 장난 아니예요😋,돼지고기구이,서울특별시 서대문구 남가좌동,22.6.27.월,고기,44,9526
116686,cnswkd,화포식당 연신내점,자주 오는 곳인데 올때마다 너무 맛있어요😋\n직원분들도 너무 친절하시고\n고기도 맛...,돼지고기구이,서울특별시 은평구 갈현동,22.5.7.토,고기,44,9526
116687,cnswkd,화포식당 가재울뉴타운점,고기 육질도 좋고 맛도 좋아요\n무엇보다 너무 잘 구워주셔서 맛있었어요😋\n소스가 ...,돼지고기구이,서울특별시 서대문구 남가좌동,22.3.27.일,고기,44,9526


In [3]:
class Args:
    pass

args = Args()
args.batch_size = 128
args.res_type = "test_v1"
args.device = "cpu"
args.dataset = "res_type"
args.dropout_rate = 0.5
args.hidden_units = 50
args.inference_only = True
args.l2_emb = 0.0
args.lr = 0.001
args.maxlen = 50
args.num_blocks = 2
args.num_epochs = 201
args.num_heads = 1
args.state_dict_path = None
args.train_dir = "test_v1"

In [7]:
dataset = data_partition(args.dataset)
[user_train, user_valid, user_test, usernum, itemnum] = dataset
sampler = WarpSampler(user_train, usernum, itemnum, batch_size=args.batch_size, maxlen=args.maxlen, n_workers=3)
model = SASRec(usernum, itemnum, args).to(args.device)

In [8]:
model

SASRec(
  (item_emb): Embedding(137, 50, padding_idx=0)
  (pos_emb): Embedding(50, 50)
  (emb_dropout): Dropout(p=0.5, inplace=False)
  (attention_layernorms): ModuleList(
    (0-1): 2 x LayerNorm((50,), eps=1e-08, elementwise_affine=True)
  )
  (attention_layers): ModuleList(
    (0-1): 2 x MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=50, out_features=50, bias=True)
    )
  )
  (forward_layernorms): ModuleList(
    (0-1): 2 x LayerNorm((50,), eps=1e-08, elementwise_affine=True)
  )
  (forward_layers): ModuleList(
    (0-1): 2 x PointWiseFeedForward(
      (conv1): Conv1d(50, 50, kernel_size=(1,), stride=(1,))
      (dropout1): Dropout(p=0.5, inplace=False)
      (relu): ReLU()
      (conv2): Conv1d(50, 50, kernel_size=(1,), stride=(1,))
      (dropout2): Dropout(p=0.5, inplace=False)
    )
  )
  (last_layernorm): LayerNorm((50,), eps=1e-08, elementwise_affine=True)
)

In [9]:
path = "/Users/sb/Desktop/modeling/SASRec.pytorch/result/res_type_test_v1/SASRec.epoch=400.lr=0.001.layer=2.head=2.hidden=50.maxlen=50.pth"
model.load_state_dict(torch.load(path))
model.eval()

SASRec(
  (item_emb): Embedding(137, 50, padding_idx=0)
  (pos_emb): Embedding(50, 50)
  (emb_dropout): Dropout(p=0.5, inplace=False)
  (attention_layernorms): ModuleList(
    (0-1): 2 x LayerNorm((50,), eps=1e-08, elementwise_affine=True)
  )
  (attention_layers): ModuleList(
    (0-1): 2 x MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=50, out_features=50, bias=True)
    )
  )
  (forward_layernorms): ModuleList(
    (0-1): 2 x LayerNorm((50,), eps=1e-08, elementwise_affine=True)
  )
  (forward_layers): ModuleList(
    (0-1): 2 x PointWiseFeedForward(
      (conv1): Conv1d(50, 50, kernel_size=(1,), stride=(1,))
      (dropout1): Dropout(p=0.5, inplace=False)
      (relu): ReLU()
      (conv2): Conv1d(50, 50, kernel_size=(1,), stride=(1,))
      (dropout2): Dropout(p=0.5, inplace=False)
    )
  )
  (last_layernorm): LayerNorm((50,), eps=1e-08, elementwise_affine=True)
)

In [12]:
u, seq, pos, neg = sampler.next_batch() # tuples to ndarray
u, seq, pos, neg = np.array(u), np.array(seq), np.array(pos), np.array(neg)


In [13]:
seq

array([[ 0,  0,  0, ...,  7, 17,  7],
       [ 0,  0,  0, ..., 36, 42,  7],
       [ 0,  0,  0, ..., 35, 42, 45],
       ...,
       [ 0,  0,  0, ..., 13, 34, 20],
       [ 0,  0,  0, ..., 25, 94,  6],
       [ 0,  0,  0, ...,  1,  1, 42]], dtype=int32)

In [18]:
seq[1]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
       13,  7, 13, 53, 13, 53, 13, 13, 13, 13, 22, 36, 36, 36, 42,  7],
      dtype=int32)